In [10]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("imjeffhi/pokemon_classifier")
model = AutoModelForImageClassification.from_pretrained("imjeffhi/pokemon_classifier")

In [11]:
import hydra
import torch
from omegaconf import DictConfig
from torch.utils.data import DataLoader
from torchvision import transforms

# Navigate to the parent directory
import os
os.chdir('..')

from src.p100.data import PokeDataset
from src.p100.model import ResNetModel


In [14]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
BUCKET_NAME = "mlops_bucket100"


In [ ]:
def evaluate(model, batch_size: int) -> None:
    """Evaluate a trained model on the test dataset."""
    print("Starting evaluation...")
    # Load the model weights
    model.eval()

    # Prepare the dataset and dataloader
    test_set = PokeDataset(bucket_name=BUCKET_NAME, mode="test", transform=transforms.ToTensor())
    test_dataloader = DataLoader(test_set, batch_size=batch_size)

    correct, total = 0, 0
    with torch.no_grad():
        for images, targets, _ in test_dataloader:
            images, targets = images.to(DEVICE), targets.to(DEVICE)
            outputs = model(images)
            predictions = outputs.argmax(dim=1)
            correct += (predictions == targets).sum().item()
            total += targets.size(0)
            print(f"Batch accuracy: {correct / total:.2%}")

    accuracy = correct / total
    print(f"Test accuracy: {accuracy:.2%}")


evaluate(model, 32)


Starting evaluation...


/Applications/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Applications/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
